In [1]:
import torch
from torch.autograd import grad
import torch.nn as nn
from numpy import genfromtxt
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd

import os
import re
import numpy as np

torch.manual_seed(123);
torch.cuda.manual_seed(123)
np.random.seed(123)
torch.backends.cudnn.deterministic = True

In [2]:
N = 6e6

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [5]:
torch.cuda.device_count()

1

In [11]:
result =[]
for i in range(299,300,1):
  print('длина волны = ', i)
  x = 190
  covid_cumulative_cases = pd.read_csv('/content/вторая волная.csv')
  covid_cumulative_cases = covid_cumulative_cases[:i]
  covid_cumulative_cases = covid_cumulative_cases.rename(columns={'Unnamed: 0': 't'})
  S = covid_cumulative_cases['S']
  I = covid_cumulative_cases['I']
  D = covid_cumulative_cases['D']
  R = covid_cumulative_cases['R']
  covid_cumulative_cases['t'] = covid_cumulative_cases['t'].astype(float)

  cumulative_susceptible = []
  cumulative_infected = []
  cumulative_dead = []
  cumulative_recovered = []
  timesteps = []

  d1 = covid_cumulative_cases['S']
  d2 = covid_cumulative_cases['I']
  d3 = covid_cumulative_cases['D']
  d4 = covid_cumulative_cases['R']
  d5 = covid_cumulative_cases['t']

  for item in range(len(d5)):
      if item % 1 == 0:
          cumulative_susceptible.append(d1[item])
          cumulative_infected.append(d2[item])
          cumulative_dead.append(d3[item])
          cumulative_recovered.append(d4[item])
          timesteps.append(d5[item])


  class DINN(nn.Module):
      def __init__(self, t, S_data, I_data, D_data, R_data):  # [t,S,I,D,R]
          super(DINN, self).__init__()
          self.N = 6e6  # population size
          self.t = torch.tensor(t, requires_grad=True).to(device)
          self.t.retain_grad()
          self.t_float = self.t.float()
          self.t_batch = torch.reshape(self.t_float, (len(self.t), 1)).to(device)
          self.S = torch.tensor(S_data).to(device)
          self.I = torch.tensor(I_data).to(device)
          self.D = torch.tensor(D_data).to(device)
          self.R = torch.tensor(R_data).to(device)

          self.losses = []
          self.save = 3  # which file to save to

          self.alpha_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))  # 0.191
          self.beta_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))  # 0.05
          self.gamma_tilda = torch.nn.Parameter(torch.rand(1, requires_grad=True))  # 0.0294

        # find values for normalization
          self.S_max = max(self.S)
          self.I_max = max(self.I)
          self.D_max = max(self.D)
          self.R_max = max(self.R)
          self.S_min = min(self.S)
          self.I_min = min(self.I)
          self.D_min = min(self.D)
          self.R_min = min(self.R)

        # unnormalize
          self.S_hat = (self.S - self.S_min) / (self.S_max - self.S_min)
          self.I_hat = (self.I - self.I_min) / (self.I_max - self.I_min)
          self.D_hat = (self.D - self.D_min) / (self.D_max - self.D_min)
          self.R_hat = (self.R - self.R_min) / (self.R_max - self.R_min)

        # matrices (x4 for S,I,D,R) for the gradients
          self.m1 = torch.zeros((len(self.t), 4)).to(device)
          self.m1[:, 0] = 1
          self.m2 = torch.zeros((len(self.t), 4)).to(device)
          self.m2[:, 1] = 1
          self.m3 = torch.zeros((len(self.t), 4)).to(device)
          self.m3[:, 2] = 1
          self.m4 = torch.zeros((len(self.t), 4)).to(device)
          self.m4[:, 3] = 1

        # NN
          self.net_sidr = self.Net_sidr()
          self.params = list(self.net_sidr.parameters())
          self.params.extend(list([self.alpha_tilda, self.beta_tilda, self.gamma_tilda]))

    # force parameters to be in a range
      @property
      def alpha(self):
          return torch.tanh(self.alpha_tilda)

      @property
      def beta(self):
          return torch.tanh(self.beta_tilda)

      @property
      def gamma(self):
          return torch.tanh(self.gamma_tilda)

    # nets
      class Net_sidr(nn.Module):  # input = [t]
          def __init__(self):
              super(DINN.Net_sidr, self).__init__()
              self.fc1 = nn.Linear(1, 200)
              self.fc2 = nn.Linear(200, 100)
              self.out = nn.Linear(100, 4)  # outputs S, I, D, R

          def forward(self, t_batch):
              sidr = F.relu(self.fc1(t_batch))
              sidr = F.tanh(self.fc2(sidr))
              sidr = self.out(sidr)
              return sidr

      def net_f(self, t_batch):
          sidr_hat = self.net_sidr(t_batch)

          S_hat, I_hat, D_hat, R_hat = sidr_hat[:, 0], sidr_hat[:, 1], sidr_hat[:, 2], sidr_hat[:, 3]

          sidr_hat.backward(self.m1, retain_graph=True)
          S_hat_t = self.t.grad.clone()
          self.t.grad.zero_()

        # I_t
          sidr_hat.backward(self.m2, retain_graph=True)
          I_hat_t = self.t.grad.clone()
          self.t.grad.zero_()

        # D_t
          sidr_hat.backward(self.m3, retain_graph=True)
          D_hat_t = self.t.grad.clone()
          self.t.grad.zero_()

        # R_t
          sidr_hat.backward(self.m4, retain_graph=True)
          R_hat_t = self.t.grad.clone()
          self.t.grad.zero_()

        # unnormalize
          S = self.S_min + (self.S_max - self.S_min) * S_hat
          I = self.I_min + (self.I_max - self.I_min) * I_hat
          D = self.D_min + (self.D_max - self.D_min) * D_hat
          R = self.R_min + (self.R_max - self.R_min) * R_hat

          f1_hat = S_hat_t - (-(self.alpha / self.N) * S * I) / (self.S_max - self.S_min)
          f2_hat = I_hat_t - ((self.alpha / self.N) * S * I - self.beta * I - self.gamma * I) / (self.I_max - self.I_min)
          f3_hat = D_hat_t - (self.gamma * I) / (self.D_max - self.D_min)
          f4_hat = R_hat_t - (self.beta * I) / (self.R_max - self.R_min)

          return f1_hat, f2_hat, f3_hat, f4_hat, S_hat, I_hat, D_hat, R_hat

      def train(self, threshold=0.0001, max_epochs=10000):
          print('\nstarting training...\n')
          epoch = 0
          while True:
            # lists to hold the output (maintain only the final epoch)
              S_pred_list = []
              I_pred_list = []
              D_pred_list = []
              R_pred_list = []

              f1, f2, f3, f4, S_pred, I_pred, D_pred, R_pred = self.net_f(self.t_batch)
              self.optimizer.zero_grad()

              S_pred_list.append(self.S_min + (self.S_max - self.S_min) * S_pred)
              I_pred_list.append(self.I_min + (self.I_max - self.I_min) * I_pred)
              D_pred_list.append(self.D_min + (self.D_max - self.D_min) * D_pred)
              R_pred_list.append(self.R_min + (self.R_max - self.R_min) * R_pred)

              loss = 0.8 * (torch.mean(torch.square(self.S_hat[:x] - S_pred[:x])) +
                         (torch.mean(torch.square(self.I_hat[:x] - I_pred[:x]))) +
                         torch.mean(torch.square(self.D_hat[:x] - D_pred[:x])) +
                         torch.mean(torch.square(self.R_hat[:x] - R_pred[:x]))) + 0.2 * (
                               torch.mean(torch.square(f1[:x])) +
                               torch.mean(torch.square(f2[:x])) +
                               torch.mean(torch.square(f3[:x])) +
                               torch.mean(torch.square(f4[:x]))
                           ) + 0.05 * torch.square(I_pred[-1] - 0)

              loss.backward()
              self.optimizer.step()
              self.scheduler.step()

              self.losses.append(loss.item())

              if loss.item() < threshold:
                  break

              if epoch % 1000 == 0:
                  print('\nEpoch ', epoch)

            # loss + model parameters update
              if epoch % 4000 == 0:
                # checkpoint save every 100 epochs if the loss is lower
                  print('\nSaving model... Loss is: ', loss)

                  print('epoch: ', epoch)
                  print('dinn.alpha', dinn.alpha)
                  print('dinn.beta', dinn.beta)
                  print('dinn.gamma', dinn.gamma)

              epoch += 1
              if epoch >= max_epochs:
                  break

          return S_pred_list, I_pred_list, D_pred_list, R_pred_list

  dinn = DINN(timesteps, cumulative_susceptible, cumulative_infected, cumulative_dead, cumulative_recovered) .to(device)

  learning_rate = 1e-5
  optimizer = optim.Adam(dinn.params, lr = learning_rate)
  dinn.optimizer = optimizer


  scheduler = torch.optim.lr_scheduler.CyclicLR(dinn.optimizer, base_lr=1e-5, max_lr=1e-3, step_size_up=1000, mode="exp_range", gamma=0.85, cycle_momentum=False)

  dinn.scheduler = scheduler

  device = next(dinn.parameters()).device

  S_pred_list, I_pred_list, D_pred_list, R_pred_list = dinn.train(threshold=0.0001, max_epochs=70000)


  res = I_pred_list[0].cpu().detach().numpy()

  result.append(res)



длина волны =  299

starting training...


Epoch  0

Saving model... Loss is:  tensor(4.1321, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
epoch:  0
dinn.alpha tensor([0.7099], device='cuda:0', grad_fn=<TanhBackward0>)
dinn.beta tensor([0.5420], device='cuda:0', grad_fn=<TanhBackward0>)
dinn.gamma tensor([0.1058], device='cuda:0', grad_fn=<TanhBackward0>)

Epoch  1000

Epoch  2000

Epoch  3000

Epoch  4000

Saving model... Loss is:  tensor(0.0541, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
epoch:  4000
dinn.alpha tensor([0.7007], device='cuda:0', grad_fn=<TanhBackward0>)
dinn.beta tensor([0.5445], device='cuda:0', grad_fn=<TanhBackward0>)
dinn.gamma tensor([0.0906], device='cuda:0', grad_fn=<TanhBackward0>)

Epoch  5000

Epoch  6000

Epoch  7000

Epoch  8000

Saving model... Loss is:  tensor(0.0401, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)
epoch:  8000
dinn.alpha tensor([0.6798], device='cuda:0', grad_fn=<TanhBackward0>)
dinn.bet

In [13]:
result

[array([ 4029.083 ,  4258.41  ,  4299.017 ,  4356.106 ,  4416.743 ,
         4572.1694,  4553.124 ,  4756.735 ,  4837.096 ,  4919.445 ,
         4972.4263,  5037.957 ,  5111.1914,  5186.2026,  5258.9805,
         5327.303 ,  5390.54  ,  5449.13  ,  5504.207 ,  5557.2725,
         5610.0684,  5664.377 ,  5721.76  ,  5783.6396,  5851.2   ,
         5925.587 ,  6007.4014,  6097.2373,  6195.275 ,  6301.875 ,
         6416.797 ,  6508.842 ,  6534.6484,  6567.894 ,  6607.8706,
         6654.115 ,  6706.1006,  6763.201 ,  6824.865 ,  6890.582 ,
         6959.7334,  7032.541 ,  7108.1104,  7186.1846,  7266.7017,
         7349.326 ,  7433.7646,  7520.43  ,  7608.687 ,  7698.6787,
         7790.334 ,  7883.6885,  7978.8325,  8075.8135,  8174.4277,
         8274.85  ,  8377.324 ,  8481.553 ,  8587.984 ,  8696.584 ,
         8767.008 ,  8818.926 ,  8873.373 ,  8930.035 ,  8988.998 ,
         9050.325 ,  9114.295 ,  9180.72  ,  9249.531 ,  9320.818 ,
         9394.803 ,  9471.195 ,  9550.15  ,  963

In [14]:
len(result)

1

In [16]:
pinn_pred = pd.DataFrame(data={
    'S': S_pred_list[0].cpu().detach().numpy(),
    'I': I_pred_list[0].cpu().detach().numpy(),
    'D': D_pred_list[0].cpu().detach().numpy(),
    'R': R_pred_list[0].cpu().detach().numpy()
})

In [18]:
pinn_pred.to_csv('./pinn_pred.csv')